# 🧠 InsightSpike-AI Google Colab Demo

**Brain-Inspired Multi-Agent Architecture for Insight Detection**

このノートブックは、InsightSpike-AIをGoogle Colab環境で実行するためのデモです。

⚡ **GPU Runtime推奨**: Runtime > Change runtime type > GPU

## 🔧 セットアップ

In [ ]:
# リポジトリのクローン
!git clone https://github.com/miyauchikazuyoshi/InsightSpike-AI.git
%cd InsightSpike-AI

In [ ]:
# 環境セットアップ（GPU対応）
# 必要なスクリプトファイルに実行権限を付与
!chmod +x scripts/colab/setup_colab.sh
!chmod +x scripts/setup/setup.sh
!chmod +x scripts/setup/refactor_prepare.sh

# Pythonスクリプトにも実行権限を付与（必要に応じて）
!find scripts/ -name "*.py" -exec chmod +x {} \;

# Colabセットアップスクリプト実行
!./scripts/colab/setup_colab.sh

In [ ]:
# セットアップ確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# 依存関係チェック
try:
    import faiss
    print(f"Faiss version: {faiss.__version__}")
except ImportError:
    print("❌ Faiss not found")

try:
    import sentence_transformers
    print(f"SentenceTransformers version: {sentence_transformers.__version__}")
except ImportError:
    print("❌ SentenceTransformers not found")

# スクリプト権限確認
print("\n🔒 Script permissions check:")
!ls -la scripts/colab/setup_colab.sh
!ls -la scripts/run_poc_simple.py

# Poetry環境確認
print("\n📬 Poetry environment check:")
!poetry --version
!poetry env info

## 📊 データ準備

In [ ]:
# データの準備（サンプルデータ作成）
# 必要なディレクトリを作成
!mkdir -p data/raw data/processed data/embedding data/models

# サンプルデータファイルを作成
with open('data/raw/test_sentences.txt', 'w', encoding='utf-8') as f:
    f.write("""The aurora borealis is caused by charged particles from the sun interacting with Earth's magnetic field.
Quantum entanglement is a phenomenon where particles become correlated in ways that defy classical physics.
Artificial intelligence uses machine learning algorithms to process data and make predictions.
The human brain contains billions of neurons that communicate through synapses.
Photosynthesis converts sunlight into chemical energy in plants.
DNA contains the genetic instructions for all living organisms.
Gravity is a fundamental force that attracts objects with mass toward each other.
Evolution explains how species change over time through natural selection.
""")

print("✅ Data directories created")
print("✅ Sample data created in data/raw/test_sentences.txt")

In [ ]:
# エピソード記憶の構築
# CLIコマンドで実行
!PYTHONPATH=src poetry run python -m insightspike.cli embed --path data/raw/test_sentences.txt

# またはMakefileを使用して実行することも可能
# !make embed

In [ ]:
# 類似性グラフの構築
!PYTHONPATH=src poetry run python -m insightspike.cli graph

## 🚀 デモ実行

In [ ]:
# 簡単なテスト実行
!PYTHONPATH=src poetry run python -m insightspike.cli loop "What is quantum entanglement?"

In [ ]:
# フルPoC実行（可視化付き）
!PYTHONPATH=src poetry run python scripts/run_poc_simple.py "How does artificial intelligence work?"

## 🔍 インタラクティブテスト

In [ ]:
# Python環境でのインタラクティブテスト
import sys
import os
sys.path.append('src')

# 環境変数設定
os.environ['PYTHONPATH'] = 'src'

from insightspike.core.layers.layer2_memory_manager import L2MemoryManager
from insightspike.core.agents.main_agent import MainAgent

# メモリとエージェントの初期化
try:
    agent = MainAgent()
    if agent.initialize():
        print("✅ Agent initialized successfully")
        memory_stats = agent.l2_memory.get_memory_stats()
        print(f"✅ Memory loaded: {memory_stats.get('total_episodes', 0)} episodes")
    else:
        print("❌ Agent initialization failed")
except Exception as e:
    print(f"❌ Setup failed: {e}")

# 質問例
question = "What causes the aurora borealis?"
print(f"Question: {question}")

# 質問処理実行
try:
    result = agent.process_question(question, verbose=True)
    print(f"\n📊 Results:")
    print(f"Response: {result.get('response', 'No response')}")
    print(f"Reasoning Quality: {result.get('reasoning_quality', 0):.3f}")
    print(f"Spike Detected: {result.get('spike_detected', False)}")
    print(f"Cycles: {result.get('total_cycles', 0)}")
except Exception as e:
    print(f"❌ Question processing failed: {e}")

## 🔧 トラブルシューティング

### よくあるエラーと解決方法

1. **CUDA not available**
   - Runtime > Change runtime type > GPU を確認

2. **ModuleNotFoundError**
   - セットアップスクリプトを再実行
   - `!pip install [missing_package]` で個別インストール

3. **FileNotFoundError: data files**
   - データ準備セクションを再実行
   - `mkdir -p data/raw` でディレクトリ作成

4. **Memory issues**
   - ランタイムを再起動: Runtime > Restart runtime

5. **Import errors**
   - `sys.path.append('src')` が正しく設定されているか確認
   - PYTHONPATH 環境変数の設定を確認